In [1]:
import os

In [2]:
%pwd

'/home/rohwid/Pacmann/lazada-id-reviews/notebooks'

In [3]:
# Change to the main directory
# So, it's executed from main directory
os.chdir("../")

In [4]:
%pwd

'/home/rohwid/Pacmann/lazada-id-reviews'

### Data Preprocessing Config

This code will be apply in `src/LadazaIDReview/entity/config_entity.py`

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataDumpConfig:
    root_dir: Path
    reviews_path: Path
    input_train_path: Path
    input_test_path: Path
    output_train_path: Path
    output_test_path: Path
    params_test_size: float

@dataclass(frozen=True)
class DataPreprocessingConfig:
    root_dir: Path
    input_train_path: Path
    input_test_path: Path
    vectorized_train_path: Path
    vectorized_test_path: Path
    model_dir: Path
    vectorizer_model_path: Path

### Data Preprocessing Config Manager

This code will be apply in `src/LazadaIDReviews/config/configurations.py`.

In [6]:
from LazadaIDReviews.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from LazadaIDReviews.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(self, 
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_dump_data_config(self) -> DataDumpConfig:
        """read data dump config file and store as config entity
        then apply the dataclasses
        
        Returns:
            config: PreprocessingConfig type
        """
        dump_config = self.config.dump_data
        ingest_config = self.config.ingest_from_sql
        dataset_params = self.params

        create_directories([dump_config.root_dir])

        config = DataDumpConfig(
            root_dir=dump_config.root_dir,
            reviews_path=ingest_config.reviews_path,
            input_train_path=dump_config.input_train_path,
            input_test_path=dump_config.input_test_path,
            output_train_path=dump_config.output_train_path,
            output_test_path=dump_config.output_test_path,
            params_test_size=dataset_params.TEST_SIZE
        )

        return config
    
    def get_preprocessing_data_config(self) -> DataPreprocessingConfig:
        """read preprocessing config file and store as config entity
        then apply the dataclasses
        
        Returns:
            config: PreprocessingConfig type
        """
        dump_config = self.config.dump_data
        vectorize_config = self.config.vectorize_data
        train_config = self.config.train_model

        create_directories([vectorize_config.root_dir])

        config = DataPreprocessingConfig(
            root_dir=vectorize_config.root_dir,
            input_train_path=Path(dump_config.input_train_path),
            input_test_path=Path(dump_config.input_test_path),
            vectorized_train_path=Path(vectorize_config.vectorized_train_path),
            vectorized_test_path=Path(vectorize_config.vectorized_test_path),
            model_dir=train_config.root_dir,
            vectorizer_model_path=Path(vectorize_config.vectorizer_model_path)
        )

        return config

### Perform Preprocessing

This code in `src/LazadaIDReview/components/preprocessing.py`.

What we would do?
+ Drop null values
+ Splitting the dataset to train and test data
+ Vectorize text using `TFIDF`

As stated before; let’s load, select columns, and drop null values from dataset.

In [8]:
import joblib
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from imblearn.over_sampling import RandomOverSampler

from LazadaIDReviews import logger

class Preprocessing:
    def __init__(self, config: DataDumpConfig):
        self.config = config

    def dump_data(self) -> None:
        """dump the splited dataset to data training and testing
        """
        logger.info(f"Read reviews file.")
        dataset_reviews = pd.read_csv(self.config.reviews_path)
        dataset = dataset_reviews[["rating", "reviewContent"]].copy()
        dataset.dropna(inplace=True)
        
        logger.info(f"Split reviews file to data train and test.")
        X_train, X_test, y_train, y_test = train_test_split(
            dataset["reviewContent"], 
            dataset["rating"], 
            test_size=self.config.params_test_size
        )
        
        # NOTE: improve the performance with ROS
        logger.info(f"Perform random over sampler.")
        ros = RandomOverSampler()
        X_train_ros, y_train_ros = ros.fit_resample(pd.DataFrame(X_train), pd.DataFrame(y_train))
        
        # NOTE: data save as series
        logger.info(f"Dump data train into {self.config.root_dir} directory.")
        X_train_ros["reviewContent"].to_pickle(self.config.input_train_path)
        X_test.to_pickle(self.config.input_test_path)
        
        # NOTE: data save as series
        logger.info(f"Dump data test into {self.config.root_dir} directory.")
        y_train_ros["rating"].to_pickle(self.config.output_train_path)
        y_test.to_pickle(self.config.output_test_path)
        
    def vectorize_data(self) -> None:
        """vectorize the splited dataset and dump vectorizer model
        """
        vectorizer = TfidfVectorizer()
        
        logger.info(f"Load data train in {self.config.input_train_path}.")
        X_train = joblib.load(self.config.input_train_path)
        
        logger.info(f"Load data test in {self.config.input_test_path}.")
        X_test = joblib.load(self.config.input_test_path)
        
        logger.info(f"Vectorize the data.")
        X_train_vec = vectorizer.fit_transform(X_train)
        X_test_vec = vectorizer.transform(X_test)
        
        logger.info(f"Dump the vectorized data.")
        joblib.dump(X_train_vec, self.config.vectorized_train_path)
        joblib.dump(X_test_vec, self.config.vectorized_test_path)
        
        logger.info(f"Creating {self.config.model_dir} directory.")
        model_dir = str(self.config.model_dir)
        os.makedirs(model_dir, exist_ok=True)
        
        logger.info(f"Save the vectorizer model.")
        joblib.dump(vectorizer, self.config.vectorizer_model_path)  

### Dump the Data Train and Data Test

This code in `src/LazadaIDReview/pipeline/step_02_preprocessing.py`.

In [9]:
try:
    config = ConfigurationManager()
    dump_data_config = config.get_dump_data_config()
    data_ingestion = Preprocessing(config=dump_data_config)
    data_ingestion.dump_data()
except Exception as e:
    logger.error(e)
    raise e

[2024-07-03 22:00:10,797: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-07-03 22:00:10,806: INFO: common: yaml file: metrics/params.yaml loaded successfully]
[2024-07-03 22:00:10,808: INFO: common: created directory at: artifacts]
[2024-07-03 22:00:10,809: INFO: common: created directory at: artifacts/data]
[2024-07-03 22:00:10,811: INFO: 2608593555: Read reviews file.]
[2024-07-03 22:00:11,294: INFO: 2608593555: Split reviews file to data train and test.]
[2024-07-03 22:00:11,303: INFO: 2608593555: Perform random over sampler.]
[2024-07-03 22:00:11,319: INFO: 2608593555: Dump data train into artifacts/data directory.]
[2024-07-03 22:00:11,364: INFO: 2608593555: Dump data test into artifacts/data directory.]


**Debug**: Read data

In [10]:
X_train = joblib.load(dump_data_config.input_train_path)
X_train

0        persen kemarin sore ...jam 2 siang datang...ga...
1        dah sampai... nyobanya nunggu pulang kerja... ...
2                                     Recommended seller..
3        Pengiriman cepat sekali 2 hari sampai, packing...
4        iish..keren pisaan.. kuy laen beli.. gak akan ...
                               ...                        
83550    Produk sesuai dengan yang ditawarkan Kendala h...
83551                 fast respon 2hari smpe...tv nya bgus
83552                              sesuai deskripsi produk
83553                                                bagus
83554                          Compatible  buat linux ga ?
Name: reviewContent, Length: 83555, dtype: object

In [11]:
X_train.isnull().sum()

0

In [12]:
y_train = joblib.load(dump_data_config.output_train_path)
y_train

0        5
1        5
2        5
3        5
4        5
        ..
83550    4
83551    4
83552    4
83553    4
83554    4
Name: rating, Length: 83555, dtype: int64

In [13]:
X_test = joblib.load(dump_data_config.input_test_path)
X_test

186968    mantap. untuk harga sejutaan kualitas nya dah ...
142884    Recomended bgt, barang cepat sampai dan sesuai...
73046                 kere barang sesuai pick thanks lazada
144710                                      Barang sesuai..
18941                                                   Oke
                                ...                        
73140                          bagus....saya suka saya suka
141077    sementara ini laptopnya belum bisa nyala, info...
143147                                               mantap
122529    Bagi Yang Minat Order/Pesan Barang Kami,Silahk...
123389                     Pengiriman cepat & packing rapih
Name: reviewContent, Length: 85624, dtype: object

In [14]:
X_test.isnull().sum()

0

In [15]:
y_test = joblib.load(dump_data_config.output_test_path)
y_test

186968    5
142884    5
73046     5
144710    5
18941     5
         ..
73140     5
141077    3
143147    5
122529    5
123389    5
Name: rating, Length: 85624, dtype: int64

### Vectorize the Data Train and Data Test

This code in `src/LazadaIDReview/pipeline/step_02_preprocessing.py`.

In [16]:
try:
    config = ConfigurationManager()
    preprocessing_config = config.get_preprocessing_data_config()
    preprocessing = Preprocessing(config=preprocessing_config)
    preprocessing.vectorize_data()
except Exception as e:
    logger.error(e)
    raise e

[2024-07-03 22:00:36,554: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-07-03 22:00:36,557: INFO: common: yaml file: metrics/params.yaml loaded successfully]
[2024-07-03 22:00:36,558: INFO: common: created directory at: artifacts]
[2024-07-03 22:00:36,561: INFO: common: created directory at: artifacts/preprocessing]
[2024-07-03 22:00:36,562: INFO: 2608593555: Load data train in artifacts/data/X_train.pkl.]
[2024-07-03 22:00:36,642: INFO: 2608593555: Load data test in artifacts/data/X_test.pkl.]
[2024-07-03 22:00:36,809: INFO: 2608593555: Vectorize the data.]
[2024-07-03 22:00:38,394: INFO: 2608593555: Dump the vectorized data.]
[2024-07-03 22:00:38,434: INFO: 2608593555: Creating artifacts/models directory.]
[2024-07-03 22:00:38,436: INFO: 2608593555: Save the vectorizer model.]


**Debug**: Read data

In [17]:
X_train_vec = joblib.load(preprocessing_config.vectorized_train_path)
X_train_vec

<83555x13584 sparse matrix of type '<class 'numpy.float64'>'
	with 1228030 stored elements in Compressed Sparse Row format>

In [18]:
X_test_vec = joblib.load(preprocessing_config.vectorized_test_path)
X_test_vec

<85624x13584 sparse matrix of type '<class 'numpy.float64'>'
	with 982605 stored elements in Compressed Sparse Row format>